In [1]:
import pandas as pd
import csv
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import gensim
from gensim import corpora, models
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from pprint import pprint

stemmer = SnowballStemmer("english") # the stopwords 
def remain_english(line):
    rule =re.compile("[^a-z^A-Z]")
    line = rule.sub('',line)
    return line
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) # remain problems

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(str(text)) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
            result.append(remain_english(lemmatize_stemming((token))).lower()) # take out not english and return into verb, and lower it 
    return result

d:\python\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
# read file into pandas
import pandas as pd

fileGP = r'C:\Users\86137\Desktop\reviews\Google Play\Reviews\GamesReview.csv'
fileApple = r'C:\Users\86137\Desktop\reviews\Apple\Reviews\apple-games-reviews.csv'

df1 = pd.read_csv(fileGP)
df2 = pd.read_csv(fileApple)
# take the target subcate reviews (Apple and GP)

dfSimulationGP = df1[df1.appSubGener.str.contains('Simulation')] # the Simulation games from GP
dfPuzzleGP = df1[df1.appSubGener.str.contains('Puzzle')] # the Puzzle games from GP
dfSimulationAp = df2[df2.genres.str.contains('Simulation')] # the Simulation games from Apple 
dfPuzzleAp = df2[df2.genres.str.contains('Puzzle')] # the Puzzle games from Apple 

# Simulation Part

In [11]:
# analyse Simulation part
df1 = dfSimulationGP
df2 = dfSimulationAp
gpNum = dfSimulationGP.count()['text']
ApNum = dfSimulationAp.count()['text'] # text(reviews) number from apple
print('targe subcate reviews num of GP and Apple:',gpNum,ApNum)

sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in df1.score.index:
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in df2.score.index:
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      
                        
print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)


targe subcate reviews num of GP and Apple: 18591 263994
number of reviews text: 282585
number of neg: 63477
number of inconsistant neg reviews text(actually a pos reviews): 404
number of pos: 184938
number of inconsistant pos reviews text(actually an neg reviews): 433


In [12]:
# negtive LDA  reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 63477
Fitting Time: 4129.480318784714
Topic: 0 
Words: 0.039*"game" + 0.039*"watch" + 0.027*"diamond" + 0.023*"time" + 0.023*"stori" + 0.022*"play" + 0.021*"energi" + 0.019*"gem" + 0.018*"wait" + 0.016*"want"
Topic: 1 
Words: 0.128*"game" + 0.061*"play" + 0.051*"time" + 0.040*"crash" + 0.023*"load" + 0.018*"start" + 0.018*"screen" + 0.017*"open" + 0.014*"delet" + 0.013*"tri"
Topic: 2 
Words: 0.025*"ball" + 0.021*"shoot" + 0.020*"time" + 0.019*"control" + 0.010*"plane" + 0.010*"go" + 0.009*"oppon" + 0.009*"lose" + 0.009*"right" + 0.008*"plan"
Topic: 3 
Words: 0.107*"game" + 0.051*"like" + 0.020*"bore" + 0.019*"look" + 0.018*"good" + 0.018*"hate" + 0.016*"stupid" + 0.015*"thing" + 0.014*"graphic" + 0.014*"play"
Topic: 4 
Words: 0.085*"game" + 0.022*"player" + 0.020*"play" + 0.012*"team" + 0.010*"develop" + 0.009*"gameplay" + 0.007*"great" + 0.007*"good" + 0.006*"better" + 0.006*""
Topic: 5 
Words: 0.051*"updat" + 0.048*"work" + 0.029*"version" + 0.025*"con

In [13]:
# positive LDA reviews analyse
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 184938
Fitting Time: 9816.025436639786
Topic: 0 
Words: 0.198*"game" + 0.082*"love" + 0.053*"like" + 0.044*"play" + 0.029*"awesom" + 0.023*"think" + 0.022*"best" + 0.018*"amaz" + 0.016*"addict" + 0.015*"cool"
Topic: 1 
Words: 0.042*"version" + 0.027*"like" + 0.024*"car" + 0.023*"monster" + 0.023*"tabl" + 0.021*"ball" + 0.019*"drive" + 0.018*"pinbal" + 0.014*"truck" + 0.013*"race"
Topic: 2 
Words: 0.058*"game" + 0.040*"money" + 0.033*"level" + 0.024*"spend" + 0.021*"like" + 0.019*"time" + 0.018*"coin" + 0.017*"thing" + 0.015*"need" + 0.015*"play"
Topic: 3 
Words: 0.019*"button" + 0.015*"turn" + 0.014*"say" + 0.014*"glitch" + 0.014*"fix" + 0.014*"need" + 0.013*"know" + 0.012*"screen" + 0.011*"peopl" + 0.011*"connect"
Topic: 4 
Words: 0.038*"like" + 0.020*"plan" + 0.017*"simul" + 0.016*"real" + 0.013*"realist" + 0.012*"control" + 0.011*"hors" + 0.011*"flight" + 0.011*"train" + 0.010*"land"
Topic: 5 
Words: 0.042*"player" + 0.026*"like" + 0.021*"prize" + 0.019*"team" + 0.016*"we

# The Puzzle Part

In [14]:
# analyse Arcade part
df1 = dfPuzzleGP
df2 = dfPuzzleAp

gpNum = dfPuzzleGP.count()['text']
ApNum = dfPuzzleAp.count()['text'] # text(reviews) number from apple
print('targe subcate reviews num of GP and Apple:',gpNum,ApNum)

sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in df1.score.index:
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in df2.score.index:
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      
                        
print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)

targe subcate reviews num of GP and Apple: 6504 263363
number of reviews text: 269867
number of neg: 43178
number of inconsistant neg reviews text(actually a pos reviews): 241
number of pos: 199926
number of inconsistant pos reviews text(actually an neg reviews): 376


In [15]:
# negtive LDA  reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 43178
Fitting Time: 1778.4108712673187
Topic: 0 
Words: 0.086*"game" + 0.047*"money" + 0.020*"pay" + 0.017*"purchas" + 0.017*"play" + 0.014*"spend" + 0.014*"free" + 0.014*"want" + 0.013*"develop" + 0.013*"peopl"
Topic: 1 
Words: 0.079*"level" + 0.068*"game" + 0.036*"play" + 0.022*"time" + 0.017*"spend" + 0.017*"money" + 0.012*"like" + 0.012*"need" + 0.010*"live" + 0.010*"complet"
Topic: 2 
Words: 0.085*"coin" + 0.031*"daili" + 0.027*"earn" + 0.025*"purchas" + 0.024*"card" + 0.023*"" + 0.021*"reward" + 0.019*"gold" + 0.019*"free" + 0.017*"prize"
Topic: 3 
Words: 0.093*"game" + 0.051*"updat" + 0.045*"play" + 0.031*"time" + 0.028*"crash" + 0.022*"work" + 0.018*"freez" + 0.017*"open" + 0.017*"screen" + 0.017*"love"
Topic: 4 
Words: 0.136*"version" + 0.045*"upgrad" + 0.021*"free" + 0.020*"charact" + 0.014*"dont" + 0.013*"stone" + 0.012*"lame" + 0.011*"pay" + 0.011*"hate" + 0.011*"better"
Topic: 5 
Words: 0.025*"game" + 0.022*"support" + 0.022*"facebook" + 0.0

In [16]:
# positive LDA reviews analyse
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 199926
Fitting Time: 5475.417810916901
Topic: 0 
Words: 0.058*"free" + 0.054*"game" + 0.038*"version" + 0.029*"review" + 0.023*"worth" + 0.021*"like" + 0.019*"peopl" + 0.017*"purchas" + 0.016*"download" + 0.015*"think"
Topic: 1 
Words: 0.158*"level" + 0.059*"game" + 0.028*"challeng" + 0.026*"play" + 0.020*"hard" + 0.019*"beat" + 0.019*"like" + 0.015*"easi" + 0.015*"get" + 0.012*"score"
Topic: 2 
Words: 0.122*"game" + 0.080*"like" + 0.039*"love" + 0.028*"awesom" + 0.022*"better" + 0.020*"cool" + 0.019*"thing" + 0.018*"think" + 0.016*"cute" + 0.013*"good"
Topic: 3 
Words: 0.108*"game" + 0.055*"challeng" + 0.048*"great" + 0.045*"think" + 0.037*"word" + 0.037*"good" + 0.028*"love" + 0.024*"make" + 0.023*"puzzl" + 0.023*"easi"
Topic: 4 
Words: 0.085*"game" + 0.025*"puzzl" + 0.022*"graphic" + 0.018*"great" + 0.016*"music" + 0.012*"simpl" + 0.011*"stori" + 0.011*"design" + 0.010*"sound" + 0.010*"look"
Topic: 5 
Words: 0.057*"puzzl" + 0.033*"like" + 0.025*"color" + 0.020*"love" + 0.